In [1]:
import ee
import geopandas as gpd

/Users/ruthparajo/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
ee.Authenticate()
ee.Initialize()

In [3]:
yell = gpd.read_file('../datos/YELL_tracts/YELL_boundary.shp')

In [4]:
def getAOI_latlon(dat,crs=4326):
    minx,miny,maxx,maxy=dat.to_crs(crs).geometry.total_bounds
    xy=[[minx,miny],[maxx,miny],[maxx,maxy],[minx,maxy]]
    aoi = ee.Geometry.Polygon(xy)
    return(aoi)

In [6]:
yell.crs

<Geographic 2D CRS: GEOGCS["North_American_Datum_1983",DATUM["North_Am ...>
Name: North_American_Datum_1983
Axis Info [ellipsoidal]:
- lon[east]: Longitude (Degree)
- lat[north]: Latitude (Degree)
Area of Use:
- undefined
Datum: North American Datum 1983
- Ellipsoid: Geodetic Reference System of 1980
- Prime Meridian: Greenwich

In [5]:
aoi=getAOI_latlon(yell)
aoi

ee.Geometry({
  "functionInvocationValue": {
    "functionName": "GeometryConstructors.Polygon",
    "arguments": {
      "coordinates": {
        "constantValue": [
          [
            [
              -111.15598590795724,
              44.1324468509431
            ],
            [
              -109.82418013760002,
              44.1324468509431
            ],
            [
              -109.82418013760002,
              45.10895671616003
            ],
            [
              -111.15598590795724,
              45.10895671616003
            ]
          ]
        ]
      },
      "evenOdd": {
        "constantValue": true
      }
    }
  }
})

In [25]:
def calculate_indices(image):
    ndvi = image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')
    mndwi = image.normalizedDifference(['SR_B3', 'SR_B7']).rename('MNDWI')  # CORREGIDO aquí
    return image.addBands([ndvi, mndwi])


In [26]:
start_year = 1986
end_year = 2020
def get_summer_image(year):
    start = ee.Date.fromYMD(year, 6, 1)
    end = ee.Date.fromYMD(year, 8, 31)
    
    # Elige Landsat según el año
    l5 = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2').filterDate(start, end)
    l7 = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2').filterDate(start, end)
    
    collection = ee.ImageCollection(ee.Algorithms.If(year <= 2011, l5, l7)) \
        .filterBounds(aoi) \
        .filter(ee.Filter.lt('CLOUD_COVER', 30)) \
        .sort('CLOUD_COVER')  # Mejor imagen (menos nubes) al inicio

    image = ee.Image(collection.first())
    image = calculate_indices(image).select(['NDVI', 'MNDWI'])
    return image.set('year', year)

# Lista de imágenes por año
years = list(range(start_year, end_year + 1))
images = [get_summer_image(y) for y in years]

# Convertir a colección
image_collection = ee.ImageCollection(images)

# Imprimir los IDs para ver si funcionó
ids = image_collection.aggregate_array('system:id').getInfo()
for i in ids:
    print(i)

LANDSAT/LT05/C02/T1_L2/LT05_037029_19860827
LANDSAT/LT05/C02/T1_L2/LT05_037029_19870830
LANDSAT/LT05/C02/T1_L2/LT05_038029_19880722
LANDSAT/LT05/C02/T1_L2/LT05_037029_19890718
LANDSAT/LT05/C02/T1_L2/LT05_039029_19900703
LANDSAT/LT05/C02/T1_L2/LT05_037030_19910825
LANDSAT/LT05/C02/T1_L2/LT05_037030_19920827
LANDSAT/LT05/C02/T1_L2/LT05_039029_19930711
LANDSAT/LT05/C02/T1_L2/LT05_037029_19940630
LANDSAT/LT05/C02/T1_L2/LT05_037029_19950820
LANDSAT/LT05/C02/T1_L2/LT05_037030_19960822
LANDSAT/LT05/C02/T1_L2/LT05_039028_19970722
LANDSAT/LT05/C02/T1_L2/LT05_037029_19980828
LANDSAT/LT05/C02/T1_L2/LT05_039028_19990712
LANDSAT/LT05/C02/T1_L2/LT05_039028_20000714
LANDSAT/LT05/C02/T1_L2/LT05_039028_20010802
LANDSAT/LT05/C02/T1_L2/LT05_038029_20020814
LANDSAT/LT05/C02/T1_L2/LT05_038029_20030801
LANDSAT/LT05/C02/T1_L2/LT05_037029_20040812
LANDSAT/LT05/C02/T1_L2/LT05_038029_20050721
LANDSAT/LT05/C02/T1_L2/LT05_039029_20060715
LANDSAT/LT05/C02/T1_L2/LT05_039029_20070718
LANDSAT/LT05/C02/T1_L2/LT05_0370

In [36]:
for image in images:
    year = image.get('year').getInfo()
    task = ee.batch.Export.image.toDrive(
        image=image.clip(aoi),
        description=f'NDVI_MNDWI_{year}',
        folder='GEE_Yellowstone',
        fileNamePrefix=f'ndvi_mndwi_{year}_summer',
        region=aoi.geometries().getInfo()[0]['coordinates'],
        scale=30,
        maxPixels=1e10
    )
    task.start()
    print(f'Task for year {year} started.')

Task for year 1986 started.
Task for year 1987 started.
Task for year 1988 started.
Task for year 1989 started.
Task for year 1990 started.
Task for year 1991 started.
Task for year 1992 started.
Task for year 1993 started.
Task for year 1994 started.
Task for year 1995 started.
Task for year 1996 started.
Task for year 1997 started.
Task for year 1998 started.
Task for year 1999 started.
Task for year 2000 started.
Task for year 2001 started.
Task for year 2002 started.
Task for year 2003 started.
Task for year 2004 started.
Task for year 2005 started.
Task for year 2006 started.
Task for year 2007 started.
Task for year 2008 started.
Task for year 2009 started.
Task for year 2010 started.
Task for year 2011 started.
Task for year 2012 started.
Task for year 2013 started.
Task for year 2014 started.
Task for year 2015 started.
Task for year 2016 started.
Task for year 2017 started.
Task for year 2018 started.
Task for year 2019 started.
Task for year 2020 started.
